Idea: find images that are adversarial for humans using a black-box attack, and see what happens when you show adversarial examples to machines.

 # Adversarial Examples that Fool both Computer Vision and Time-Limited Humans

Some adversarial examples for neural networks fool resource-limited humans ([Elsayed et al. 2018](https://arxiv.org/abs/1802.08195)). You present an image to a human and ask them to categorize the image.

![Experiment setup](w3-assets/experiment_setup.png)

With this setup, they show that images that are adversarially trained to fool artificial neural networks can also fool humans. There are a few tricks to make the adversarial images work on humans:

* the adversial images are found from an ensemble of networks (more robust examples)
* backward masking makes the task harder so humans aren't near saturation
* artifical retina concentrates the effect near fixation point where acuity is highest in humans
    
However, the effects are very small - 51-53% choices towards the adversarial category:

<img src="w3-assets/class_probability.png" width="250"/>




# Towards Deep Learning Models Resistant to Adversarial Attacks

https://arxiv.org/pdf/1706.06083.pdf

Important observation: classification loss and adversarial risk are two very different things that a classifier can be optimized for. 

*Classification loss*: the goal of classification is minimize the empirical risk (e.g. cross-entropy loss) over the data function:

$$\mathbb{E}_{(x,y)\sim D} L(x,y,\theta)$$

Where $x$ are examplars, $y$ are labels, $L$ is the classification loss, $\theta$ are the network parameters, and $D$ is the data distribution. 

On the other hand, the *adversarial risk* has to do with the likelihood of misclassifying a data point within a neighborhood $S$ of a data point $x$. $S$ might be the $L_\infty$ ball, for example. So the first loss has to do with the data distribution, the other with points that are outside the data distribution.

To combine these two notions, we try to find a set of parameters $\theta$ that minimizes the risk:

$$\mathbb{E}_{(x,y)\sim D} \max_{\delta \in S} L(x+\delta,y,\theta)$$

This has an interesting minimax form that is studied in the paper.

*PGD*: Projected gradient descent. Do gradient descent with respect to the loss and project back into the $S$ neighborhood ball. See explanation here: https://math.stackexchange.com/a/572664/583479

Section 3 fizzles with empirical investigations of the structure of the loss function, but makes an argument that PGD is the master first-order method. They make an argument that a robust set of weights is found by replacing examples with PGD based adversarial examples.

Higher capacity models are more robust to adversarial attacks because they can learn more complicated boundaries.

Iterated PGD is a best-in-class adversarial attack method for white box problems.

# Adversarial Risk and the Dangers of Evaluating Against Weak Attacks

https://arxiv.org/pdf/1802.05666.pdf

> *Abstract*: This paper investigates recently proposed approaches for defending against adversarial examples and evaluating adversarial robustness. The existence of adversarial examples in trained neural networks reflects the fact that expected risk alone does not capture the model's performance against worst-case inputs. We motivate the use of adversarial risk as an objective, although it cannot easily be computed exactly. We then frame commonly used attacks and evaluation metrics as defining a tractable surrogate objective to the true adversarial risk. This suggests that models may be obscured to adversaries, by optimizing this surrogate rather than the true adversarial risk. We demonstrate that this is a significant problem in practice by repurposing gradient-free optimization techniques into adversarial attacks, which we use to decrease the accuracy of several recently proposed defenses to near zero. Our hope is that our formulations and results will help researchers to develop more powerful defenses.

> We note two potential explanations for models with high evaluation performance:  models which learn a robust decision boundary, and thus admit few adversarial examples, and models which achieve security via obscurity.

Is it possible that humans have are not really robust, but hide their non-robustness via obscurity? What if we try to open up the box?

Rather than finding an adversary according to the true cross-entropy, they use a surrogate loss:

$$J(x) = \text{logit}_i(x) - \max_{j\ne i}\text{logit}_j(x)$$

Here $i$ is the label of the real example $x_0$. This surrogate loss is easier to optimize than the cross-entropy, and has the right properties, e.g. it's negative when $x$ is an adversarial example.

To perform black-box optimization, they use the SPSA algorithm, which works by randomly perturbing the input and obtaining an estimated gradient. One could construct a similar method where one would instead approximate the local function as a plane, take random samples on that plane, and solve for the maximum likelihood plane under a condition of small slope. Their algorithm looks like this:

<img src="w3-assets/spsa_attack.png" width="500" />

They also point out two other methods: natural evolutionary strategies and zero-order optimization.



# Synthesizing robust adversarial examples

https://arxiv.org/pdf/1707.07397.pdf

Adversarial examples can be fragile to rotations, scaling, etc. The authors thus propose generating robust adversarial examples under continuous transformations $T$. They use a convex relaxation of the usual objective of keeping the distance between the original and the adversarial example small. 

Specifically, the problem is formulated as finding an example $x'$ close to $x$ following transformations in the class $T$ that maximizes the probability of the wrong class $y$:

$$\arg \max_{x'} \left( \mathbb{E}_{t \sim T}(\log p(y|t(x')) - \lambda \mathbb{E}_{t\sim T}(d(t(x'), t(x)) \right)$$

The problem can be solved by:

* replacing the expectations with specific transformations
* having the transformations be continuous with respect to $x$

For affine transformations (2D), this is straightforward. For 3d transformations, one has to remember the nonlinear coordinate coordinate transformations through rendering. One could do similar things with lighting, etc.

Where it's relevant for our problem is that human observers naturally apply stochastic transformations, e.g. they saccade, they're at different positions from the screen, etc. Therefore, if we wanted to build adversarial examples from humans, we have to take into account that they'd have to work from different angles/persepectives.

# Black-box Adversarial Attacks with Limited Queries and Information

https://arxiv.org/pdf/1804.08598.pdf

This paper uses Natural Evolution Strategies (NES) to attack Inception V3 and the Google Cloud Vision API in the black box setting. They consider three different settings:

* query-limited: you get the full $p(y|x)$ from the model
* partial-information: you get scores (which have some unclear relationship to $p(y|x)$ for the top 5 or top 10 labels for a given image
* label-only: you get only the top k labels, no scores
  
Remarkably, breaking the models can be accomplished in as little 10k to 40 k evaluations for the query limited and partial information cases.

The natural evolutionary strategy starts with the premise of an expected loss over a distribution:

$$\mathbb{E}_{x'\sim \pi(x, x')} L(x') = \int F(x')\pi(x'|x)dx'$$

The idea behind NES is to take find the gradient of this quantity - using the familiar score trick, we find that:

$$\nabla \mathbb{E} [L] = \mathbb{E}_{x' \sim \pi(x, x')}\left[ L(x') \nabla \log \pi(x, x')\right]$$

If we choose the proposal distribution to be the iid normal distribution centered around $x$, it follows that the expression reduces to:

$$\nabla \mathbb{E} [L] = 1/\sigma^2 \mathbb{E}_{x' \sim \pi(x, x')}\left[ L(x') x'\right]$$

To reduce the variance of this estimate, they propose sampling pairs at a time, $x' = \{x-\delta, x+\delta\}$. 

The obtained estimate looks suspiciously like the white noise Wiener-Volterra first-order kernel (Marmarelis & Marmarelis 1973). The twist with the variance reduction is clever, however.

In the partial information and label only cases, they start from the target class than the starting class.

# Other references

Loose ends: classification images.

* [Obfuscated gradients give a false sense of security](https://www.anishathalye.com/media/2018/07/19/poster.pdf): lots of ways of breaking an adversarial defense. If we think that humans have adversarial defenses, maybe we can break them with some of these techniques. Most of the techniques quoted are for white-box methods, i.e. methods where gradients are computable. However, there are many references in this poster that are also relevant to black box attacks.